In [1]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis, mode, binom, poisson, norm, expon, f_oneway, ttest_ind, ttest_rel
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.anova import AnovaRM
import stat_240829 as stat

# Descriptive Statistics: describe()

In [2]:
np.random.seed(42)
df = pd.DataFrame({
    'Var1': np.random.rand(20),
    'Var2': np.random.rand(20),
    'Var3': np.random.rand(20),
    'Var4': np.random.rand(20),
    'Var5': np.random.rand(20),
    'c': ['X', 'X', 'Y', 'Z', 'Y', 'X', 'Z', 'Y', 'Z', 'Y','A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'Y']
})

In [3]:
stat.describe(df,cols=['Var1','Var2','Var3','Var4','Var5'])

,mean,median,variance,std_dev,mad,min,max,range,skewness,kurtosis,count,sum,25%,50%,75%
Var1,0.457702,0.403243,0.094671,0.307687,0.234321,0.020584,0.969910,0.949325,0.277863,-1.263481,20,9.154046,0.183010,0.403243,0.714053
Var2,0.454829,0.448111,0.084785,0.291179,0.242280,0.046450,0.965632,0.919182,0.227964,-1.084160,20,9.096579,0.192386,0.448111,0.629948
Var3,0.489971,0.507622,0.109425,0.330794,0.317204,0.034389,0.969585,0.935196,0.116779,-1.421157,20,9.799421,0.193201,0.507622,0.805056
Var4,0.461058,0.373572,0.098833,0.314378,0.299274,0.005522,0.986887,0.981365,0.098072,-1.460762,20,9.221165,0.184268,0.373572,0.771514
Var5,0.487343,0.508264,0.072195,0.268691,0.201131,0.025419,0.887213,0.861794,-0.279719,-1.048767,20,9.746863,0.321633,0.508264,0.717335


# Statistical Test Flow Chart: test()

In [ ]:
### Revising because this is too much ###
''' test: Statistical test flow chart
        question: difference, correlation, categories
        factors: # of factors
        same: same (True) or different (False) subjects
        compare: list of comparisons
        para: parameteric (True) or nonparametric (False)
    Dependencies:
'''
def test(df: pd.DataFrame(), compare: list(), question: str(), factors=1, same=False, para=True):
    return ''

need to reformat to tidy data and then it will be good for pairwise turkey_hsd

In [4]:
''' difference: Statistical test flow chart
        df: tidy DataFrame
        data_col: data column name
        compare_col: comparisons column name
        compare: list of comparisons
        group: column name to split tidy dataframes (optional)
        factors: # of factors
        same: same (True) or different (False) subjects
        compare: list of comparisons
        para: parameteric (True) or nonparametric (False)
        alpha: significance level for the test (Default: 0.05)
    Dependencies:
'''
def difference(df: pd.DataFrame(), data_col: str(), compare_col: str(), compare: list(), factors=1, same=False, para=True, alpha=0.05):

    if factors==1: # quantity of factors (condition 2 levels?)

        if not same: # Different samples

            if para==True: # Data that follows a normal distribution

                if len(compare)==2: # Comparing 2 samples

                    # Student's T Test Calculation
                    print('Statistical Test: Student\'s T-test \n - Only use if you have two groups or if you are comparing just two of the n groups and are not concerned about inflating the Type I error rate (e.g., False Positives).')
                    t_stat, p_value = ttest_ind(*(df[df[compare_col]==comp][data_col] for comp in compare))
                    inference = pd.DataFrame({'test':['Student\'s T-test']*2,
                                              'comparison': [','.join(compare)]*2, 
                                              'statistic': ['t_stat','p_value'],
                                              'value':[t_stat,p_value]})
                    
                    return inference
                
                elif len(compare)>2: # Comparing more than 2 samples
                    
                    # 1-way Anova Test
                    print('Statistical Test: 1-way Anova\n - If you want to compare all three groups simultaneously to determine if there is a significant difference in their means.\n - If the ANOVA finds a significant difference, then perform post-hoc tests to determine which specific groups differ from each other.')
                    f_stat, p_value = f_oneway(*(df[df[compare_col]==comp][data_col] for comp in compare))
                    if p_value<alpha: null='reject'
                    else: null='accept'
                    inference = pd.DataFrame({'test':['1-way Anova Test']*2,
                                              'comparison': [','.join(compare)]*2, 
                                              'statistic': ['f_stat','p_value'],
                                              'value':[f_stat,p_value],
                                              'null': ['',null]})
                    
                    # Tukey's Honestly Significant Difference Test
                    print('Statistical Test: Tukey\'s Honestly Significant Difference (HSD) Test\n - It\'s suitable when you have equal or unequal group sizes.\n - Other options include Bonferroni Correction, Scheffé\'s Test, Dunnett\'s Test, and Holm\'s Sequential Bonferroni Procedure.')
                    df2 = pd.concat([df[df[compare_col]==comp] for comp in compare]).reset_index(drop=True) # Only include specified data
                    tukey_result = pairwise_tukeyhsd(df2[data_col], df2[compare_col], alpha=alpha)
                    tukey_df = pd.DataFrame(data=tukey_result._results_table.data[1:], columns=tukey_result._results_table.data[0])
                    inference = pd.concat([inference,
                                           pd.DataFrame({'test':['Tukey\'s HSD Test']*len(compare),
                                                         'comparison': [','.join([tukey_df.iloc[i]['group1'],tukey_df.iloc[i]['group2']]) for i in range(len(tukey_df))], 
                                                         'statistic': ['p-adj']*len(compare),
                                                         'value': tukey_df['p-adj']})]).reset_index(drop=True)

                    return inference
                
                else: print('Error: Invalid compare. List needs to contain 2 or more stings')
            
            else: # Data that does not follow a normal distribution
                print('Statistical Test: Mann Whitney U Test\nWIP...')
        
        else: # Same sample

            if para==True:  # Data that follows a normal distribution

                if len(compare)==2: # Comparing 2 related (paired) samples
                    
                    # Student's Paired T-test
                    print('Statistical Test: Paired Student\'s T-test \n  - Only use if you have two groups or if you are comparing just two of the n groups and are not concerned about inflating the Type I error rate (e.g., False Positives).\n - Assumes the two groups are related or paired, meaning that each data point in one group has a corresponding data point in the other group.')
                    t_stat, p_value = ttest_rel(*(df[df[compare_col]==comp][data_col] for comp in compare))
                    inference = pd.DataFrame({'test':['Paired Student\'s T-test']*2,
                                              'comparison': [','.join(compare)]*2, 
                                              'statistic': ['t_stat','p_value'],
                                              'value':[t_stat,p_value]})
                    
                    return inference

                elif len(compare)>2: # Comparing 2 or more related (paired) samples
                    
                    # Repeated Anova
                    print('Statistical Test: Repeated 1-way Anova\nWIP...')
                    df2 = pd.concat([df[df[compare_col]==comp] for comp in compare]).reset_index(drop=True) # Only include specified data
                    anova = AnovaRM(data=df2, depvar=data_col, subject=compare_col)
                    anova_result = anova.fit()
                    print(anova_result)

                else: print('Error: Invalid compare. List needs to contain 2 or more stings')

            else: 
                print('Statistical Test: Wilcocson Paired Test\nWIP...')
    
    # Deal with later...
    elif factors>1: print('Statistical Test: 2 way anova, General Linear (Mixed) Model, etc.\nNot Included...')
    else: print('Error: Invalid factors. Number needs to be greater or equal to 1.')

In [66]:
df

,Var1,Var2,Var3,Var4,Var5,c
0,0.374540,0.611853,0.122038,0.388677,0.863103,X
1,0.950714,0.139494,0.495177,0.271349,0.623298,X
2,0.731994,0.292145,0.034389,0.828738,0.330898,Y
3,0.598658,0.366362,0.909320,0.356753,0.063558,Z
4,0.156019,0.456070,0.258780,0.280935,0.310982,Y
5,0.155995,0.785176,0.662522,0.542696,0.325183,X
6,0.058084,0.199674,0.311711,0.140924,0.729606,Z
7,0.866176,0.514234,0.520068,0.802197,0.637557,Y
8,0.601115,0.592415,0.546710,0.074551,0.887213,Z
9,0.708073,0.046450,0.184854,0.986887,0.472215,Y


In [65]:
AnovaRM?

Init signature:
AnovaRM(
    data,
    depvar,
    subject,
    within=None,
    between=None,
    aggregate_func=None,
)
Docstring:     
Repeated measures Anova using least squares regression

The full model regression residual sum of squares is
used to compare with the reduced model for calculating the
within-subject effect sum of squares [1].

Currently, only fully balanced within-subject designs are supported.
Calculation of between-subject effects and corrections for violation of
sphericity are not yet implemented.

Parameters
----------
data : DataFrame
depvar : str
    The dependent variable in `data`
subject : str
    Specify the subject id
within : list[str]
    The within-subject factors
between : list[str]
    The between-subject factors, this is not yet implemented
aggregate_func : {None, 'mean', callable}
    If the data set contains more than a single observation per subject
    and cell of the specified model, this function will be used to
    aggregate the data before r

In [71]:
difference(df=df,data_col='Var5',compare_col='c',compare=['X','Y'],factors=1,same=False,para=True)

Statistical Test: Student's T-test 
 - Only use if you have two groups or if you are comparing just two of the n groups and are not concerned about inflating the Type I error rate (e.g., False Positives).


,test,comparison,statistic,value
0,Student's T-test,"X,Y",t_stat,1.418112
1,Student's T-test,"X,Y",p_value,0.205944


In [72]:
difference(df=df,data_col='Var5',compare_col='c',compare=['X','Y','Z'],factors=1,same=False,para=True)

Statistical Test: 1-way Anova
 - If you want to compare all three groups simultaneously to determine if there is a significant difference in their means.
 - If the ANOVA finds a significant difference, then perform post-hoc tests to determine which specific groups differ from each other.
Statistical Test: Tukey's Honestly Significant Difference (HSD) Test
 - It's suitable when you have equal or unequal group sizes.
 - Other options include Bonferroni Correction, Scheffé's Test, Dunnett's Test, and Holm's Sequential Bonferroni Procedure.


,test,comparison,statistic,value
0,1-way Anova Test,"X,Y,Z",f_stat,0.721473
1,1-way Anova Test,"X,Y,Z",p_value,0.515145
2,Tukey's HSD Test,"X,Y",p-adj,0.547800
3,Tukey's HSD Test,"X,Z",p-adj,0.981700
4,Tukey's HSD Test,"Y,Z",p-adj,0.665900


In [73]:
difference(df=df,data_col='Var5',compare_col='c',compare=['A','B'],factors=1,same=True,para=True)

Statistical Test: Paired Student's T-test 
  - Only use if you have two groups or if you are comparing just two of the n groups and are not concerned about inflating the Type I error rate (e.g., False Positives).
 - Assumes the two groups are related or paired, meaning that each data point in one group has a corresponding data point in the other group.


,test,comparison,statistic,value
0,Paired Student's T-test,"A,B",t_stat,-0.378255
1,Paired Student's T-test,"A,B",p_value,0.741616


In [68]:
difference(df=df,data_col='Var5',compare_col='c',compare=['A','B','C'],factors=1,same=True,para=True)

Statistical Test: Repeated 1-way Anova
WIP...


TypeError: argument of type 'NoneType' is not iterable

In [7]:
df

,Var1,Var2,Var3,Var4,Var5,c
0,0.374540,0.020584,0.611853,0.607545,0.122038,X
1,0.950714,0.969910,0.139494,0.170524,0.495177,X
2,0.731994,0.832443,0.292145,0.065052,0.034389,Y
3,0.598658,0.212339,0.366362,0.948886,0.909320,Y
4,0.156019,0.181825,0.456070,0.965632,0.258780,Y
5,0.155995,0.183405,0.785176,0.808397,0.662522,X
6,0.058084,0.304242,0.199674,0.304614,0.311711,X
7,0.866176,0.524756,0.514234,0.097672,0.520068,Y
8,0.601115,0.431945,0.592415,0.684233,0.546710,Y
9,0.708073,0.291229,0.046450,0.440152,0.184854,Y


In [65]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd
pairwise_tukeyhsd?

Signature: pairwise_tukeyhsd(endog, groups, alpha=0.05)
Docstring:
Calculate all pairwise comparisons with TukeyHSD confidence intervals

Parameters
----------
endog : ndarray, float, 1d
    response variable
groups : ndarray, 1d
    array with groups, can be string or integers
alpha : float
    significance level for the test

Returns
-------
results : TukeyHSDResults instance
    A results class containing relevant data and some post-hoc
    calculations, including adjusted p-value

Notes
-----
This is just a wrapper around tukeyhsd method of MultiComparison

See Also
--------
MultiComparison
tukeyhsd
statsmodels.sandbox.stats.multicomp.TukeyHSDResults
File:      ~/anaconda3/lib/python3.11/site-packages/statsmodels/stats/multicomp.py
Type:      function

In [67]:
pairwise_tukeyhsd(endog=data[['Var1','Var2','Var3','Var4','Var5']],groups=['Var1','Var2','Var3','Var4','Var5'])

ValueError: data has 10 elements and groups has 5

In [ ]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Assuming you have a DataFrame df with a column 'values' and a column 'group'
tukey_result = pairwise_tukeyhsd(data['values'], data['group'], alpha=0.05)
print(tukey_result)

Post-hoc tests are used after a one-way ANOVA when you find a statistically significant difference between the group means. These tests help you determine which specific groups differ from each other. Here are some common post-hoc tests used after a one-way ANOVA:

1. Tukey's Honestly Significant Difference (HSD) Test
Purpose: The most commonly used post-hoc test, Tukey's HSD, compares all possible pairs of group means to identify which ones differ significantly.
Usage: It's suitable when you have equal or unequal group sizes.
Implementation:
In Python, you can use pairwise_tukeyhsd from statsmodels.stats.multicomp.
python
Copy code
from statsmodels.stats.multicomp import pairwise_tukeyhsd

# Assuming you have a DataFrame df with a column 'values' and a column 'group'
tukey_result = pairwise_tukeyhsd(df['values'], df['group'], alpha=0.05)
print(tukey_result)
2. Bonferroni Correction
Purpose: Adjusts the significance level to account for the number of comparisons being made, reducing the risk of Type I errors.
Usage: You apply it when performing multiple pairwise t-tests. It's a more conservative approach.
Implementation:
You would manually adjust the p-value threshold by dividing the original alpha level by the number of comparisons.
3. Scheffé's Test
Purpose: A very conservative test, Scheffé's test can be used when comparing complex comparisons (not just pairwise). It's suitable when you suspect non-pairwise differences.
Usage: It's less commonly used due to its conservative nature.
Implementation: It's available in some statistical software, but not directly implemented in popular Python libraries.
4. Dunnett's Test
Purpose: Compares multiple treatment groups against a single control group.
Usage: Ideal when you have one control group and want to compare it against several other groups.
Implementation: Specialized packages or functions may be required for this in Python.
5. Holm’s Sequential Bonferroni Procedure
Purpose: A step-down procedure that adjusts p-values to control the family-wise error rate. It's less conservative than the Bonferroni correction.
Usage: Useful when dealing with multiple comparisons.
Implementation: You can use multipletests from statsmodels.
python
Copy code
from statsmodels.stats.multitest import multipletests

# Assuming pvals is a list or array of p-values from multiple tests
pvals_corrected = multipletests(pvals, method='holm')
print(pvals_corrected)
6. Dunn’s Test
Purpose: A non-parametric test used when the ANOVA assumptions are violated. It is used after a Kruskal-Wallis test (non-parametric alternative to ANOVA).
Usage: Suitable when you have non-normally distributed data.
Implementation: The scikit-posthocs package can be used for Dunn's test in Python.

In [57]:
stats.f_oneway(data[['Var1','Var2','Var3','Var4','Var5']])

TypeError: at least two inputs are required; got 1.

In [59]:
data

,Var1,Var2,Var3,Var4,Var5,c
0,0.374540,0.020584,0.611853,0.607545,0.122038,X
1,0.950714,0.969910,0.139494,0.170524,0.495177,X
2,0.731994,0.832443,0.292145,0.065052,0.034389,Y
3,0.598658,0.212339,0.366362,0.948886,0.909320,Y
4,0.156019,0.181825,0.456070,0.965632,0.258780,Y
5,0.155995,0.183405,0.785176,0.808397,0.662522,X
6,0.058084,0.304242,0.199674,0.304614,0.311711,X
7,0.866176,0.524756,0.514234,0.097672,0.520068,Y
8,0.601115,0.431945,0.592415,0.684233,0.546710,Y
9,0.708073,0.291229,0.046450,0.440152,0.184854,Y


In [62]:
compare = ['Var1','Var2']
t_stat, p_value = stats.ttest_ind(data[compare[0]],data[compare[1]])
pd.DataFrame({'t_stat': t_stat, 'p_value': p_value},index=pd.Index([','.join(compare)],name='compare')).reset_index()

,compare,t_stat,p_value
0,"Var1,Var2",0.903049,0.378427


In [64]:
compare = ['Var1','Var2','Var3']
f_stat, p_value = stats.f_oneway(*(data[col] for col in compare))
pd.DataFrame({'f_stat': f_stat, 'p_value': p_value},index=pd.Index([','.join(compare)],name='compare')).reset_index()

,compare,f_stat,p_value
0,"Var1,Var2,Var3",0.609184,0.5511


In [54]:
stats.f_oneway?

Signature: stats.f_oneway(*samples, axis=0)
Docstring:
Perform one-way ANOVA.

The one-way ANOVA tests the null hypothesis that two or more groups have
the same population mean.  The test is applied to samples from two or
more groups, possibly with differing sizes.

Parameters
----------
sample1, sample2, ... : array_like
    The sample measurements for each group.  There must be at least
    two arguments.  If the arrays are multidimensional, then all the
    dimensions of the array must be the same except for `axis`.
axis : int, optional
    Axis of the input arrays along which the test is applied.
    Default is 0.

Returns
-------
statistic : float
    The computed F statistic of the test.
pvalue : float
    The associated p-value from the F distribution.

Warns
-----
`~scipy.stats.ConstantInputWarning`
    Raised if all values within each of the input arrays are identical.
    In this case the F statistic is either infinite or isn't defined,
    so ``np.inf`` or ``np.nan`` is retu

In [53]:
stats.ttest_ind?

Signature:
stats.ttest_ind(
    a,
    b,
    axis=0,
    equal_var=True,
    nan_policy='propagate',
    permutations=None,
    random_state=None,
    alternative='two-sided',
    trim=0,
    *,
    keepdims=False,
)
Docstring:
Calculate the T-test for the means of *two independent* samples of scores.

This is a test for the null hypothesis that 2 independent samples
have identical average (expected) values. This test assumes that the
populations have identical variances by default.

Parameters
----------
a, b : array_like
    The arrays must have the same shape, except in the dimension
    corresponding to `axis` (the first, by default).
axis : int or None, default: 0
    If an int, the axis of the input along which to compute the statistic.
    The statistic of each axis-slice (e.g. row) of the input will appear in a
    corresponding element of the output.
    If ``None``, the input will be raveled before computing the statistic.
equal_var : bool, optional
    If True (default), pe